In [ ]:
import pandas as pd
# import cosine similarity function
from sklearn.metrics.pairwise import cosine_similarity

In [105]:
chefmozaccepts = pd.read_csv('./data_set/chefmozaccepts.csv')
chefmozcuisine = pd.read_csv('./data_set/chefmozcuisine.csv')
chefmozhours4 = pd.read_csv('./data_set/chefmozhours4.csv')
chefmozparking = pd.read_csv('./data_set/chefmozparking.csv')
geoplaces2 = pd.read_csv('./data_set/geoplaces2.csv')
rating_final = pd.read_csv('./data_set/rating_final.csv')
usercuisine = pd.read_csv('./data_set/usercuisine.csv')
userpayment = pd.read_csv('./data_set/userpayment.csv')
userprofile = pd.read_csv('./data_set/userprofile.csv')

### Collaborative Filtering With Cosine Similarity
User-to-User Collaborative Filtering
memory based
__1__ Cosine Similarity

In [106]:
duplicated_value = rating_final[['userID', 'placeID']].duplicated().sum()
print("there is:", duplicated_value, "duplicated_value")
# for each userID just exist one placeID, Because of that there is no need to mean or weighted mean to doing so, or deleting extra record

there is: 0 duplicated_value


In [107]:
# Make rating matrix
user_place_df = rating_final.pivot(columns='placeID', index='userID', values='rating')
# Fill Nan value with zero, indicate user do not rate it
user_place_df.fillna(0, inplace=True)
# Print first 5 row
user_place_df.head()

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
U1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
U1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [108]:
# Create user similarity matrix, 
user_sim_matrix = cosine_similarity(user_place_df)
# Create user similarity data frame
user_sim_df = pd.DataFrame(
    user_sim_matrix,
    index=user_place_df.index, 
    columns=user_place_df.index, # used index attribute, we calculated user pairwise similarity
)
# Print first 5 record
user_sim_df.head()

userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
userID,,,,,,,,,,,,,,,,,,,,,
U1001,1.000000,0.227921,0.166957,0.000000,0.059761,0.111456,0.188982,0.0,0.106904,0.000000,...,0.0,0.0,0.0,0.353553,0.0,0.083478,0.0,0.000000,0.148250,0.000000
U1002,0.227921,1.000000,0.266371,0.158362,0.095346,0.088911,0.075378,0.0,0.426401,0.000000,...,0.0,0.0,0.0,0.402911,0.0,0.199778,0.0,0.322329,0.413919,0.355335
U1003,0.166957,0.266371,1.000000,0.000000,0.000000,0.325645,0.000000,0.0,0.374817,0.000000,...,0.0,0.0,0.0,0.118056,0.0,0.439024,0.0,0.059028,0.476463,0.208232
U1004,0.000000,0.158362,0.000000,1.000000,0.166091,0.077440,0.131306,0.0,0.037139,0.000000,...,0.0,0.0,0.0,0.350931,0.0,0.000000,0.0,0.280745,0.103005,0.000000
U1005,0.059761,0.095346,0.000000,0.166091,1.000000,0.000000,0.237171,0.0,0.000000,0.447214,...,0.0,0.0,0.0,0.084515,0.0,0.000000,0.0,0.000000,0.124035,0.000000


In [109]:
# Define function for select top 5 user_ids similar to given an user_id
def fetch_sim_users(user_place_data, user_id, k=5) -> pd.Series:
    """This function separates the selected user from all
        other users and then takes a cosine similarity of 
        the selected user with all users to find similar 
        users. Return the top k similar users (by CustomerID)
        to our selected user

    Args:
        user_place_data (pandas DataFrame): user-place-matrix 

        user_id (str): user identification

        k (int, optional): Top K similar users. Defaults to 5.
    """

    # Separating data rows for the entered user id
    user_place_record = user_place_data[user_place_data.index == user_id]
    # a data of all other users 
    other_users_place_records = user_place_data[user_place_data.index != user_id]
    # calculate cosine similarity between user and each other user
    sim_score = cosine_similarity(user_place_record, other_users_place_records)[0].tolist()
    # users indices
    user_indices = other_users_place_records.index
    
    # Use pandas Series data structure, 
    sim_ser = pd.Series(data=sim_score, index=user_indices, name=f'Users are similar to {user_id}')
    # Sort from most similar to least similar
    sim_ser.sort_values(ascending=False, inplace=True)
    # Return top k users similar to the specific user_
    return sim_ser.iloc[:k]
# Let's find top 5 similar users to U1001 user   
fetch_sim_users(user_place_data=user_place_df, user_id = 'U1001', k=5)

userID
U1036    0.417392
U1054    0.417392
U1092    0.404061
U1116    0.397033
U1055    0.395437
Name: Users are similar to U1001, dtype: float64